In [9]:
import random
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Set a random seed for reproducibility
random.seed(42)
np.random.seed(42)

# ==============================================================================
# 1. CORE MARKET SIMULATION (UNCHANGED)
#    - Order ID Generator
#    - Order Class
#    - LimitOrderBook Class
# ==============================================================================

def gen_order_id():
    """
    A generator function to produce a stream of unique order IDs.
    Each ID is a string in the format 'O{i}'.
    """
    i = 0
    while True:
        yield f"O{i}"
        i += 1

_order_id_gen = gen_order_id()

class Order:
    """
    Represents a single order in the limit order book.
    (This class is identical to your LOB.ipynb)
    """
    def __init__(self, trader_id, side, price, size, order_type='limit', timestamp=0):
        """
        Initializes an Order object.
        """
        self.id = next(_order_id_gen)
        self.trader_id = trader_id
        self.side = side  # 'bid' or 'ask'
        self.price = price  # integer ticks or None for market
        self.size = size
        self.remaining = size
        self.type = order_type  # 'limit' or 'market'
        self.timestamp = timestamp

class LimitOrderBook:
    """
    Implements a limit order book (LOB) for a single financial asset.
    It handles order placing, matching, and cancellation.
    (This class is identical to your LOB.ipynb)
    """

    def __init__(self, mid_price=10000, tick_size=1, max_levels=100):
        self.mid_price = mid_price
        self.tick = tick_size
        self.bids = defaultdict(deque)
        self.asks = defaultdict(deque)
        self.bids_prices = set()
        self.asks_prices = set()
        self.order_map = {}  # order_id -> (order, container)
        self.trade_history = []  # (time, price, size, taker_id, maker_id)
        self.time = 0
        self.max_levels = max_levels
        # Per-trader accounting (inventory, cash)
        self.trader_inventory = defaultdict(int)
        self.trader_cash = defaultdict(float)

    def best_bid(self):
        """Returns the highest bid price currently in the book."""
        return max(self.bids_prices) if self.bids_prices else None

    def best_ask(self):
        """Returns the lowest ask price currently in the book."""
        return min(self.asks_prices) if self.asks_prices else None

    def mid(self):
        """Calculates the current mid-price of the market."""
        bb = self.best_bid()
        ba = self.best_ask()
        if bb is None and ba is None:
            return self.mid_price
        if bb is None:
            return ba - self.tick
        if ba is None:
            return bb + self.tick
        return (bb + ba) / 2.0

    def place_limit_order(self, order: Order):
        """
        Places a limit order in the book. If the order is marketable (crosses the spread),
        it will be matched against existing orders. Otherwise, it's added to the book.
        """
        if order.side == 'bid':
            if self.best_ask() is not None and order.price >= self.best_ask():
                self._match_order(order)
                if order.remaining > 0:
                    self._add_to_book(order)
            else:
                self._add_to_book(order)
        else:
            if self.best_bid() is not None and order.price <= self.best_bid():
                self._match_order(order)
                if order.remaining > 0:
                    self._add_to_book(order)
            else:
                self._add_to_book(order)

    def place_market_order(self, side, size, trader_id):
        """Places a market order, which is immediately matched against the best available orders."""
        order = Order(trader_id=trader_id, side=side, price=None, size=size, order_type='market', timestamp=self.time)
        self._match_order(order)
        return order

    def _add_to_book(self, order):
        """A helper function to add a limit order to the appropriate side of the book."""
        container = self.bids if order.side == 'bid' else self.asks
        prices_set = self.bids_prices if order.side == 'bid' else self.asks_prices
        
        # Ensure price is a hashable type (int)
        if not isinstance(order.price, int):
             # This check can help debug if a non-int price gets here
             print(f"Warning: Non-integer price in _add_to_book: {order.price} type {type(order.price)}")
             # Attempt to cast, though the error should be fixed upstream
             try:
                 order.price = int(order.price)
             except Exception as e:
                 print(f"Failed to cast price: {e}")
                 return # Don't add order

        container[order.price].append(order)
        prices_set.add(order.price)
        self.order_map[order.id] = (order, container)
        # trim far levels
        if len(prices_set) > self.max_levels:
            if order.side == 'bid':
                far = min(prices_set)
                while container[far]:
                    o = container[far].popleft()
                    self.order_map.pop(o.id, None)
                del container[far]
                prices_set.remove(far)
            else:
                far = max(prices_set)
                while container[far]:
                    o = container[far].popleft()
                    self.order_map.pop(o.id, None)
                del container[far]
                prices_set.remove(far)

    def _match_order(self, taker_order: Order):
        """
        Matches a 'taker' order against existing 'maker' orders in the book.
        This function handles the logic for trade execution and updating accounts.
        """
        if taker_order.side == 'bid':
            opposite = self.asks
            opp_prices = sorted(self.asks_prices)
            price_cmp = lambda p: p <= taker_order.price if taker_order.price is not None else True
        else:
            opposite = self.bids
            opp_prices = sorted(self.bids_prices, reverse=True)
            price_cmp = lambda p: p >= taker_order.price if taker_order.price is not None else True

        for p in opp_prices:
            if taker_order.price is not None and not price_cmp(p):
                break
            queue = opposite[p]
            while queue and taker_order.remaining > 0:
                maker = queue[0]
                trade_size = min(maker.remaining, taker_order.remaining)
                trade_price = maker.price if maker.price is not None else p
                # record
                self.trade_history.append((self.time, trade_price, trade_size,
                                           taker_order.trader_id if taker_order.side=='bid' else maker.trader_id,
                                           maker.trader_id if taker_order.side=='bid' else taker_order.trader_id))
                # update accounting: buyer inventory +, buyer cash -, seller inventory -, seller cash +
                if taker_order.side == 'bid':
                    buyer = taker_order.trader_id
                    seller = maker.trader_id
                else:
                    buyer = maker.trader_id
                    seller = taker_order.trader_id
                
                # apply cash/inventory transfer at trade_price
                self.trader_inventory[buyer] += trade_size
                self.trader_inventory[seller] -= trade_size
                self.trader_cash[buyer] -= trade_price * trade_size
                self.trader_cash[seller] += trade_price * trade_size

                maker.remaining -= trade_size
                taker_order.remaining -= trade_size
                if maker.remaining == 0:
                    queue.popleft()
                    self.order_map.pop(maker.id, None)
                if not queue:
                    if taker_order.side == 'bid':
                        self.asks_prices.discard(p)
                        if p in self.asks: del self.asks[p]
                    else:
                        self.bids_prices.discard(p)
                        if p in self.bids: del self.bids[p]
                if taker_order.remaining == 0:
                    break

    def cancel_order(self, order_id):
        """Removes an order from the book by its ID."""
        if order_id not in self.order_map:
            return False
        order, container = self.order_map.pop(order_id)
        q = container[order.price]
        newq = deque([o for o in q if o.id != order_id])
        if q: # Only update if q is not empty
            container[order.price] = newq
        if not newq:
            if container is self.bids:
                self.bids_prices.discard(order.price)
                if order.price in self.bids: del self.bids[order.price]
            else:
                self.asks_prices.discard(order.price)
                if order.price in self.asks: del self.asks[order.price]
        return True

    def step_time(self):
        self.time += 1

# ==============================================================================
# 2. TRADER AGENTS (UNCHANGED)
#    - NoiseTrader Class
#    - InformedTrader Class
# ==============================================================================

class NoiseTrader:
    """
    A simple trader agent that places random market orders.
    (This class is identical to your LOB.ipynb)
    """
    def __init__(self, trader_id, market: LimitOrderBook, rate=0.2):
        self.id = trader_id
        self.mkt = market
        self.rate = rate

    def act(self):
        if random.random() < self.rate:
            side = random.choice(['bid', 'ask'])
            size = random.randint(1, 3)
            self.mkt.place_market_order(side=side, size=size, trader_id=self.id)

class InformedTrader:
    """
    A trader agent that simulates having private information.
    (This class is identical to your LOB.ipynb)
    """
    def __init__(self, trader_id, market: LimitOrderBook, info_prob=0.05, strength=4):
        self.id = trader_id
        self.mkt = market
        self.info_prob = info_prob
        self.strength = strength

    def act(self, signal=None):
        """
        Acts based on a signal.
        """
        if signal is None:
            if random.random() < self.info_prob:
                signal = random.choice([1, -1])
            else:
                return
        if signal == 1:
            self.mkt.place_market_order(side='bid', size=self.strength, trader_id=self.id)
        elif signal == -1:
            self.mkt.place_market_order(side='ask', size=self.strength, trader_id=self.id)

# ==============================================================================
# 3. MARKET MAKER ENVIRONMENT (MODIFIED)
#    - Added `mm_active` flag to __init__
#    - Added `agent_data_log` and `market_data_log` for data collection
#    - Modified `reset()` to clear logs and log initial state
#    - Modified `step()` to:
#      - Respect `mm_active` flag (MM only acts if True)
#      - Call new logging functions at each step
#    - Added `_log_agent_data()` and `_log_market_data()` methods
# ==============================================================================

class MarketMakerEnv:
    """
    The reinforcement learning environment for the market maker agent.
    This class defines the state, actions, and rewards for the agent.
    (Based on the PPO version in your LOB.ipynb, cell 90066d55)
    
    --- NEW FEATURES ---
    - `mm_active` flag: Toggle the market maker's participation.
    - `agent_data_log`: Logs inventory and wealth for ALL agents at each step.
    - `market_data_log`: Logs mid-price, spread, and depth at each step.
    """

    def __init__(self,
                 mid_price=1000,
                 tick=1,
                 top_k=1, # <-- FIX 1: Changed from 5 to 1
                 agent_id="MM",
                 num_noise=100,
                 seed_liquidity=10,
                 mm_active=True): # <-- NEW: Flag to control MM actions
        
        self.lob = LimitOrderBook(mid_price=mid_price, tick_size=tick, max_levels=200)
        self.agent_id = agent_id
        self.top_k = top_k
        self.seed_liquidity = seed_liquidity
        
        # --- BUGFIX: Store agent params to recreate them in reset() ---
        self.num_noise = num_noise
        self.noise_rate = 0.25
        self.informed_info_prob = 0.05
        self.informed_strength = 6
        # --- End Bugfix ---
        
        # Create traders (they will be recreated in reset)
        self.noise_traders = [NoiseTrader(f"N{i}", self.lob, rate=self.noise_rate) for i in range(self.num_noise)]
        self.informed = InformedTrader("I1", self.lob, info_prob=self.informed_info_prob, strength=self.informed_strength)
        self.last_agent_orders = []
        
        # --- NEW: Data logging and MM activity flag ---
        self.mm_active = mm_active
        self.agent_data_log = []
        self.market_data_log = []
        # --- End New ---
        
        self.reset()
        
        # Define observation and action space (for stable-baselines)
        # This will now be (1*4 + 2) = 6, matching the model
        self.action_space = np.array([16]) # 16 discrete actions
        self.observation_space = np.array([self._get_obs().shape[0]]) # obs shape

    def reset(self):
        """
        Resets the environment to a new, clean state.
        """
        # --- BUGFIX: Create new LOB and RE-CREATE all traders ---
        # This ensures all traders point to the *same* new LOB
        self.lob = LimitOrderBook(mid_price=self.lob.mid_price, tick_size=self.lob.tick, max_levels=self.lob.max_levels)
        self.noise_traders = [NoiseTrader(f"N{i}", self.lob, rate=self.noise_rate) for i in range(self.num_noise)]
        self.informed = InformedTrader("I1", self.lob, info_prob=self.informed_info_prob, strength=self.informed_strength)
        # --- End Bugfix ---

        mid = self.lob.mid_price
        for p in range(mid-10, mid):
            o = Order(trader_id="seed", side='bid', price=p, size=random.randint(self.seed_liquidity-5, self.seed_liquidity+5),
                      order_type='limit', timestamp=0)
            self.lob.place_limit_order(o)
        for p in range(mid+1, mid+11):
            o = Order(trader_id="seed", side='ask', price=p, size=random.randint(self.seed_liquidity-5, self.seed_liquidity+5),
                      order_type='limit', timestamp=0)
            self.lob.place_limit_order(o)
            
        self.lob.trader_inventory = defaultdict(int)
        self.lob.trader_cash = defaultdict(float)
        self.lob.trader_inventory[self.agent_id] = 0
        self.lob.trader_cash[self.agent_id] = 0.0
        self.last_agent_orders = []
        self.steps = 0
        
        # --- NEW: Clear logs and log initial state (step 0) ---
        self.agent_data_log = []
        self.market_data_log = []
        self._log_market_data()
        self._log_agent_data()
        # --- End New ---
        
        self.prev_wealth = 0.0 # Agent's wealth for reward calc
        return self._get_obs()

    def _get_obs(self):
        """
        Constructs the observation (state) vector for the RL agent.
        """
        # --- FIX 2: Changed from (top_k * 4 + 1) to (top_k * 4 + 2) ---
        # k bids (price, vol), k asks (price, vol), 1 inventory, 1 cash
        obs = np.zeros(self.top_k * 4 + 2) 
        
        bids = sorted(self.lob.bids_prices, reverse=True)[:self.top_k]
        asks = sorted(self.lob.asks_prices)[:self.top_k]
        mid = self.lob.mid()

        for i in range(self.top_k):
            if i < len(bids):
                price = bids[i]
                size = sum([o.remaining for o in self.lob.bids[price]])
                obs[i*2] = (mid - price) / self.lob.tick # price offset
                obs[i*2 + 1] = size
        
        for i in range(self.top_k):
            if i < len(asks):
                price = asks[i]
                size = sum([o.remaining for o in self.lob.asks[price]])
                obs[self.top_k*2 + i*2] = (price - mid) / self.lob.tick # price offset
                obs[self.top_k*2 + i*2 + 1] = size

        # --- FIX 2: Add inventory AND cash to match trained model ---
        obs[-2] = self.lob.trader_inventory[self.agent_id]
        # Get cash, use .get for safety, and normalize by mid-price
        cash = self.lob.trader_cash.get(self.agent_id, 0.0)
        obs[-1] = cash / mid if mid > 0 else 0.0 
        
        return obs
    
    def _cancel_agent_orders(self):
        """Helper to cancel all of the agent's previous orders."""
        for oid in self.last_agent_orders:
            self.lob.cancel_order(oid)
        self.last_agent_orders = []

    # --- NEW: Data logging method for market state (Goal 1b) ---
    def _log_market_data(self):
        """
        Logs the current market mid-price, spread, and top-level depth.
        This fulfills your request for market data at each timestep.
        """
        bb = self.lob.best_bid()
        ba = self.lob.best_ask()
        mid_price = self.lob.mid()
        
        spread = None
        if bb is not None and ba is not None:
            spread = ba - bb
            
        depth_bid = 0
        if bb is not None and bb in self.lob.bids:
            depth_bid = sum(o.remaining for o in self.lob.bids[bb])
            
        depth_ask = 0
        if ba is not None and ba in self.lob.asks:
            depth_ask = sum(o.remaining for o in self.lob.asks[ba])

        self.market_data_log.append({
            "step": self.lob.time,
            "mid_price": mid_price,
            "spread": spread,
            "best_bid_depth": depth_bid,
            "best_ask_depth": depth_ask
        })
    # --- End New ---

    # --- NEW: Data logging method for all agents (Goal 1a) ---
    def _log_agent_data(self):
        """
        Logs inventory, cash, and wealth for ALL agents at the current step.
        This fulfills your request for agent data at each timestep.
        """
        mid_price = self.lob.mid()
        
        # Get all unique trader IDs from both cash and inventory
        trader_ids = set(self.lob.trader_inventory.keys()) | set(self.lob.trader_cash.keys())
        
        # Include traders who haven't acted yet but are in the env
        trader_ids.add(self.agent_id)
        trader_ids.add(self.informed.id)
        for nt in self.noise_traders:
            trader_ids.add(nt.id)
        
        for trader_id in trader_ids:
            if trader_id == "seed": continue # Optional: ignore seed trader
            
            inventory = self.lob.trader_inventory.get(trader_id, 0)
            cash = self.lob.trader_cash.get(trader_id, 0.0)
            # Wealth = cash balance + value of inventory holdings at mid-price
            wealth = cash + (inventory * mid_price) 
            
            self.agent_data_log.append({
                "step": self.lob.time,
                "trader_id": trader_id,
                "inventory": inventory,
                "cash": cash,
                "wealth": wealth
            })
    # --- End New ---

    def step(self, action):
        """
        The environment advances one timestep.
        --- MODIFIED ---
        - MM action is now conditional on `self.mm_active`
        - Data logging is called at the end of the step.
        """
        self.steps += 1
        
        # --- MODIFIED: MM only acts if this flag is True ---
        if self.mm_active:
            # Action *must* be an int, not a numpy array
            if not isinstance(action, int):
                try:
                    action = int(action)
                except Exception as e:
                    print(f"Warning: Could not cast action {action} to int: {e}")
                    action = 0 # Default to a safe action
            
            # Action: 16 options -> (bid_offset 0-3, ask_offset 0-3)
            bid_off = (action // 4) + 1  # 1..4 ticks away
            ask_off = (action % 4) + 1  # 1..4 ticks away
            size = 1 # Fixed size from your notebook
            
            # cancel previous agent orders
            self._cancel_agent_orders()

            mid = int(self.lob.mid())
            bid_price = mid - bid_off
            ask_price = mid + ask_off

            # place agent limit orders
            bid_order = Order(trader_id=self.agent_id, side='bid', price=bid_price, size=size, order_type='limit', timestamp=self.lob.time)
            ask_order = Order(trader_id=self.agent_id, side='ask', price=ask_price, size=size, order_type='limit', timestamp=self.lob.time)
            self.lob.place_limit_order(bid_order)
            self.lob.place_limit_order(ask_order)
            self.last_agent_orders = [bid_order.id, ask_order.id]
        else:
            # If MM is not active, still cancel any lingering orders
            # (e.g., from seeding or a previous run)
            self._cancel_agent_orders()
        # --- End Modification ---

        # Other traders act
        for nt in self.noise_traders:
            nt.act()
        future_signal = None
        if random.random() < 0.02:
            future_signal = random.choice([1, -1])
        self.informed.act(signal=future_signal)

        # step time
        self.lob.step_time()

        # compute reward: delta wealth (cash + inventory*mid) minus penalty
        mid_now = self.lob.mid()
        wealth = self.lob.trader_cash[self.agent_id] + self.lob.trader_inventory[self.agent_id] * mid_now
        reward = wealth - self.prev_wealth
        inv = self.lob.trader_inventory[self.agent_id]
        inv_penalty = -0.1 * (inv**2) # quadratic inventory penalty
        reward += inv_penalty
        self.prev_wealth = wealth

        # --- NEW: Log data at the end of the step ---
        self._log_market_data()
        self._log_agent_data()
        # --- End New ---

        obs = self._get_obs()
        done = self.steps >= 500 # Episode horizon
        
        info = {
            "wealth": wealth,
            "inventory": inv,
            "mid": mid_now,
            "trade_count": len(self.lob.trade_history)
        }
        
        return obs, reward, done, info

# ==============================================================================
# 4. SIMULATION RUNNER SCRIPT (FIXED)
#    - This script fulfills your experimental design:
#      - Loads your pre-trained PPO model
#      - Defines a `run_simulation` function
#      - Runs 50 simulations WITH the Market Maker
#      - Runs 50 simulations WITHOUT the Market Maker
#      - Saves all collected data into four CSV files for analysis
# ==============================================================================

def run_simulation(model, timesteps=500, mm_active=True):
    """
    Helper function to run a single simulation episode.
    
    Args:
        model: The trained PPO agent (can be None if mm_active is False).
        timesteps (int): Max steps per episode.
        mm_active (bool): Whether the MM agent should place orders.

    Returns:
        (pd.DataFrame, pd.DataFrame): 
            - A DataFrame of all agent data for the episode.
            - A DataFrame of all market data for the episode.
    """
    # Create the environment, passing the mm_active flag
    env = MarketMakerEnv(mm_active=mm_active)
    obs = env.reset()
    
    for _ in range(timesteps):
        if mm_active:
            # If MM is active, use the model to predict an action
            action, _ = model.predict(obs, deterministic=True)
            
            # --------------------- THE FIX ---------------------
            # model.predict() returns a 0-dim numpy array (e.g., array(7))
            # We MUST convert it to a standard Python int (e.g., 7)
            # before passing it to env.step()
            action = int(action)
            # ------------------- END OF FIX --------------------
            
        else:
            # If MM is not active, action doesn't matter. Pass a dummy value.
            action = 0 
            
        obs, reward, done, info = env.step(action)
        
        if done:
            break
            
    # Collect the data logs from the environment
    agent_data = pd.DataFrame(env.agent_data_log)
    market_data = pd.DataFrame(env.market_data_log)
    
    return agent_data, market_data

def main():
    """
    Main function to run the full 50/50 simulation experiment.
    """
    NUM_SIMULATIONS = 50
    TIMESTEPS = 500 # Set this to your desired episode length
    
    # --- Load your trained PPO model ---
    # Make sure the "ppo_market_maker.zip" file is in the same directory
    try:
        model = PPO.load("ppo_market_maker")
    except FileNotFoundError:
        print("Error: 'ppo_market_maker.zip' not found.")
        print("Please ensure your trained model is in the same directory as this script.")
        return
    except Exception as e:
        print(f"Error loading PPO model: {e}")
        print("Please ensure stable_baselines3 is installed.")
        return

    print("--- Starting 50 simulations WITH Market Maker ---")
    
    # Lists to store DataFrames from each simulation
    all_agent_data_with_mm = []
    all_market_data_with_mm = []

    for i in range(NUM_SIMULATIONS):
        agent_df, market_df = run_simulation(model, timesteps=TIMESTEPS, mm_active=True)
        
        # Add a 'sim_id' column to track which simulation the data belongs to
        agent_df['sim_id'] = i
        market_df['sim_id'] = i
        
        all_agent_data_with_mm.append(agent_df)
        all_market_data_with_mm.append(market_df)
        print(f"Completed simulation (MM Active): {i+1}/{NUM_SIMULATIONS}")

    print("\n--- Starting 50 simulations WITHOUT Market Maker ---")
    
    all_agent_data_no_mm = []
    all_market_data_no_mm = []

    for i in range(NUM_SIMULATIONS):
        # Pass `model=None` or any dummy value, it won't be used when mm_active=False
        agent_df, market_df = run_simulation(model, timesteps=TIMESTEPS, mm_active=False)
        
        agent_df['sim_id'] = i
        market_df['sim_id'] = i
        
        all_agent_data_no_mm.append(agent_df)
        all_market_data_no_mm.append(market_df)
        print(f"Completed simulation (MM Inactive): {i+1}/{NUM_SIMULATIONS}")

    # --- Combine all data into single DataFrames and Save to CSV ---
    
    print("\nSimulations complete. Saving data to CSV files...")
    
    try:
        # Combine the lists of DataFrames into four master DataFrames
        final_agent_with_mm = pd.concat(all_agent_data_with_mm)
        final_market_with_mm = pd.concat(all_market_data_with_mm)
        final_agent_no_mm = pd.concat(all_agent_data_no_mm)
        final_market_no_mm = pd.concat(all_market_data_no_mm)
        
        # Save to CSV
        final_agent_with_mm.to_csv("agent_data_with_mm.csv", index=False)
        final_market_with_mm.to_csv("market_data_with_mm.csv", index=False)
        final_agent_no_mm.to_csv("agent_data_no_mm.csv", index=False)
        final_market_no_mm.to_csv("market_data_no_mm.csv", index=False)
        
        print("Successfully saved data to:")
        print("- agent_data_with_mm.csv")
        print("- market_data_with_mm.csv")
        print("- agent_data_no_mm.csv")
        print("- market_data_no_mm.csv")

    except Exception as e:
        print(f"Error saving data: {e}")
        print("Please ensure you have write permissions and 'pandas' is installed.")

# This makes the `main()` function run when you execute the script
if __name__ == "__main__":
    main()



--- Starting 50 simulations WITH Market Maker ---
Completed simulation (MM Active): 1/50
Completed simulation (MM Active): 2/50
Completed simulation (MM Active): 3/50
Completed simulation (MM Active): 4/50
Completed simulation (MM Active): 5/50
Completed simulation (MM Active): 6/50
Completed simulation (MM Active): 7/50
Completed simulation (MM Active): 8/50
Completed simulation (MM Active): 9/50
Completed simulation (MM Active): 10/50
Completed simulation (MM Active): 11/50
Completed simulation (MM Active): 12/50
Completed simulation (MM Active): 13/50
Completed simulation (MM Active): 14/50
Completed simulation (MM Active): 15/50
Completed simulation (MM Active): 16/50
Completed simulation (MM Active): 17/50
Completed simulation (MM Active): 18/50
Completed simulation (MM Active): 19/50
Completed simulation (MM Active): 20/50
Completed simulation (MM Active): 21/50
Completed simulation (MM Active): 22/50
Completed simulation (MM Active): 23/50
Completed simulation (MM Active): 24/50